In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","8g") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .getOrCreate()

In [11]:
import time

# get the start time
st = time.time()

surl = 'neo4j://localhost:7692'
spassword = 'password'
turl='neo4j+s://f16978a0.databases.neo4j.io'
tpassword = 'oVt8aMnDVH52GXYMTUrm81vkx2xjphHW62F4ArtNXLI'

In [12]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c:Course) return c.id as id") \
  .load()

In [13]:
df.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "id")\
    .option("labels",":Course")\
    .save()

In [14]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c1:Course)-[r1:REQUIRE_PL|REQUIRE_TOOL|REQUIRE_FRAM|REQUIRE_PLAT|REQUIRE_KNOW]->(d:Competency)<-[r2:TEACH_PL|REQUIRE_TOOL|REQUIRE_FRAM|REQUIRE_PLAT|REQUIRE_KNOW]-(c2:Course) WHERE id(c1) < id(c2) return c1.id as sourceid, c2.id as targetid") \
  .load()

In [15]:
df.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("relationship", 'TO')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','sourceid:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Course')\
    .option('relationship.target.node.keys','targetid:id')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [16]:
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 10.617608308792114 seconds


In [17]:
spark.stop()